In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.fft import rfft,rfftfreq,irfft
import requests

sns.set()

In [ ]:
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101G/ManadFoddDod'
json_post = {
  "query": [
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Manad",
      "selection": {
        "filter": "item",
        "values": [
          "01",
          "02",
          "03",
          "04",
          "05",
          "06",
          "07",
          "08",
          "09",
          "10",
          "11",
          "12"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "000000NF"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url,json = json_post)
r.status_code

In [ ]:
json = r.json()
data = json['data']

In [ ]:
dead_df = pd.DataFrame.from_dict(data)
dead_df

In [ ]:
dead_df[['gender','month','year']] = dead_df['key'].to_list()
dead_df['dead'] = dead_df['values'].apply(lambda x : x[0])
dead_df.drop(['key','values'],axis=1,inplace=True)
dead_df['dead'] = dead_df['dead'].astype(int)
dead_df = dead_df.groupby(['year','month']).sum()
dead_df

In [ ]:
dead_df = dead_df.loc['2015':'2019']
dead_df

In [ ]:
# deaths with unknown date not included # 
dead_df.groupby('year').sum()
start_year = dead_df.index[0][0]
end_year = dead_df.index[-1][0]
end_year

In [ ]:
title = 'SWEDEN monthly deaths [time domain] {} - {}'.format(start_year,end_year)
dead_df.plot(figsize=(18,12),style='o--',title=title)
plt.ylabel('nr of deaths')
plt.savefig('swe_fft_time_domain_deaths_{}_{}.jpg'.format(start_year,end_year),format='jpg')

In [ ]:
sample_rate = 12 # monthly data : set unit a year

signal = dead_df['dead']
signal = signal - signal.mean()
signal = signal.values

yf = rfft(signal) 

xf = rfftfreq(len(signal),1 / sample_rate)


In [ ]:
plt.figure(figsize=(18,12))
plt.title('SWEDEN FFT on all cause deaths [Frequency domain] for years {} - {}'.format(start_year,end_year))
plt.xlabel('frequency [1/year]')
plt.ylabel('Normalized "Power"')
plt.plot(xf,np.abs(yf) / np.abs(yf).max())
yf_max = yf.argmax()
print (yf_max)
plt.savefig('fft_swe_deaths_periodicity_{}_{}.jpg'.format(start_year,end_year),format='jpg')

In [ ]:
dominant_freq = np.zeros_like(yf)
dominant_freq[yf_max] = yf[yf_max]

plt.figure(figsize=(18,12))
plt.plot(xf,np.abs(dominant_freq))

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

yf_filtered = irfft(dominant_freq)


plt.figure(figsize=(18,12))

ax = plt.gca()

title = 'SWEDEN {} - {} death periodicity obtained from FFT compared to actual data'.format(start_year,end_year)

ax.set_title(title)
ax.plot((yf_filtered / yf_filtered.max() + 1) / (yf_filtered / yf_filtered.max() +1).max(),'o--',
        label='reverse engineered monthly deaths obtained from FFT using top frequency')
ax.plot(NormalizeData(dead_df['dead'].values),'o--',label='actual monthly deaths')

ax.set_xlabel('month nr')
ax.set_ylabel('"Power"')

ax.legend(loc='upper right')
#ax.set_xticklabels(range(1,len(dead_df) + 1),rotation=90)


#for label in ax.xaxis.get_ticklabels()[0::12]:
        #label.set_visible(False)

In [ ]:
plt.plot(xf,yf)